In [13]:
import pandas as pd
import numpy as np

In [77]:
def calc_marginal_concordances(el1, el2, info):
    cs = []
    for i, (a, b) in enumerate(zip(el1, el2)):
        t, q, p, _ = info[i].values()
        if t == 'g':
            if b < a + q:
                cs.append(1)
            elif b < a + p:
                cs.append((a+p-b)/(p-q))
            else:
                cs.append(0)
        else: # criterion type cost
            if b < a - p:
                cs.append(0)
            elif b < a - q:
                cs.append((b-a+p)/(p-q))
            else:
                cs.append(1)
    return cs

def calc_marginal_discordances(el1, el2, info):
    ds = []
    for i, (a, b) in enumerate(zip(el1, el2)):
        t, _, p, v = info[i].values()
        if t == 'g':
            if b < a + p:
                ds.append(0)
            elif b < a + v:
                ds.append((b-a-p)/(v-p))
            else:
                ds.append(1)
        else: # criterion type cost
            if b < a - v:
                ds.append(1)
            elif b < a - p:
                ds.append((a-p-b)/(v-p))
            else:
                ds.append(0)
    return ds

def comprehensive_concordance(marginal_conc, weights):
    return np.dot(np.array(weights), np.array(marginal_conc)) / np.sum(weights)

def outranking_credibilities_sigma(conc_C: float, dis_D):
    return conc_C * np.prod([(1-dis)/(1-conc_C) for dis in dis_D if dis > conc_C])

def create_credibility_mat(df, weights, info):
    arr = np.array(df)
    credibility_mat = np.zeros((arr.shape[0],arr.shape[0]))
    for i in range(arr.shape[0]):
        for j in range(arr.shape[0]):
            c = calc_marginal_concordances(arr[i], arr[j], info)
            D = calc_marginal_discordances(arr[i], arr[j], info)
            C = comprehensive_concordance(c, weights)
            credibility_mat[i,j] = outranking_credibilities_sigma(C, D)

    return credibility_mat

In [79]:
df = pd.read_csv("przyklad_wyk.csv")
weights = [2, 3]
info = [
    {'type': 'g', 'iff': 10, 'pref':50, 'veto':100},
    {'type': 'c', 'iff': 0, 'pref':10, 'veto':20} 
]
credibility_mat = create_credibility_mat(df, weights, info)
credibility_mat


array([[1. , 0. , 0. ],
       [1. , 1. , 0.7],
       [1. , 0.3, 1. ]])

In [81]:
df = pd.read_csv("przyklad_lab.csv")
weights = [3, 3, 4]
info = [
    {'type': 'g', 'iff': 4, 'pref':12, 'veto':28},
    {'type': 'g', 'iff': 1, 'pref':2, 'veto':8},
    {'type': 'c', 'iff': 100, 'pref':200, 'veto':600} 
]
credibility_mat = create_credibility_mat(df, weights, info)
credibility_mat

array([[1.        , 0.6       , 0.3       , 0.7       , 0.55      ],
       [0.        , 1.        , 0.2037037 , 0.        , 0.        ],
       [0.58333333, 0.6       , 1.        , 0.85      , 0.        ],
       [0.3       , 0.        , 0.6       , 1.        , 0.58333333],
       [0.6       , 0.        , 0.6       , 0.7       , 1.        ]])